# Titanic Example

Load libraries.

In [26]:
import pandas as pd
import numpy as np
import sys
from sklearn.model_selection import train_test_split
load("~/conjecturing/sage/conjecturing.py")

Specify output files.

In [27]:
inv_file = open("2022_12_07_inv.txt", "w")
prop_file = open("2022_12_07_prop.txt", "w")

Specify the number of examples to use for conjecturing and skips.

In [28]:
num_train = 10
my_skips = 0.3

Read data.  

In [29]:
my_data =pd.read_csv("train.csv",
                    index_col=int(0),
                    header=int(0)
                    )
print(my_data.shape)
my_data.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Check the data types of the columns.  For categorical data, make sure the type is integer or objects.  Make sure the categories do not contain special characters besides numbers and "_".

In [30]:
my_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


Pandas thinks Survived and Pclass are integers/numeric, but they are categorical.  Recast them as objects.

In [31]:
my_data = my_data.astype({"Survived": object, "Pclass": object})

Create a new feature which is the first letter of the cabin.

In [32]:
my_data["cabin_letter"]=my_data["Cabin"].str[:1]

Identify invariant and categorical columns and the target column.  The target should be in one of the lists.

In [33]:
invariant_names=["Age", "Fare", "SibSp", "Parch"] 
categorical_names=["Survived", "Sex","Pclass", "cabin_letter", "Embarked"]
target = "Survived"

Generic code starts here.  Rename target column.  Select columns.

In [34]:
if target in categorical_names:
    categorical_names[categorical_names.index(target)] = "TARGET"
else:
    invariant_names[invariant_names.index(target)] = "TARGET"

my_data = my_data.rename(columns={target: "TARGET"})
print(my_data.columns)
my_data = my_data[invariant_names + categorical_names]
print(categorical_names)

Index(['TARGET', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'cabin_letter'],
      dtype='object')
['TARGET', 'Sex', 'Pclass', 'cabin_letter', 'Embarked']


Convert categorical variables to dummies.  One dummy for each binary variable and one dummy for each level for variables with more than two levels.

In [35]:
property_names = []
for col in categorical_names:
    if col != "TARGET":
        unique_vals=list(my_data[col].unique())  # if nan is a level
        #unique_vals=list(my_data[col].dropna().unique())  # if nan is not a level
        if len(unique_vals)==2: # just use one level for binary features
            property_names.append(col+"_"+str(unique_vals[1]))
        elif len(unique_vals) > 2: #one property for each level.
            for level in unique_vals:
                property_names.append(col+"_"+str(level))


if "TARGET" in categorical_names:
    target_property_names = []
    unique_vals = list(my_data["TARGET"].unique()) # if nan is a level
    #unique_vals = list(my_data["TARGET"].dropna().unique()) # if nan is not a level
    if len(unique_vals)==2:
        target_property_names.append("TARGET_"+str(unique_vals[1]))
    elif len(unique_vals) > 2:
        for level in unique_vals:
            target_property_names.append("TARGET_"+str(level))
            
my_df = pd.get_dummies(my_data, 
                       columns=categorical_names,
                       dtype=np.uint8,
                       dummy_na=True,  # False is the default.  If False, use dropna() above
                       drop_first=False) # False is the default

my_df = my_df.rename(lambda col: col.replace('.0', ''), axis='columns')
my_df.head()

,Age,Fare,SibSp,Parch,TARGET_0,TARGET_1,TARGET_nan,Sex_female,Sex_male,Sex_nan,...,cabin_letter_D,cabin_letter_E,cabin_letter_F,cabin_letter_G,cabin_letter_T,cabin_letter_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,22.0,7.2500,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
2,38.0,71.2833,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,26.0,7.9250,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
4,35.0,53.1000,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
5,35.0,8.0500,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0


Define class, invariants, properties, and target properties (if applicable).

In [36]:
class Example():
    def __init__(self, name, mydf):
        self.name = name
        self.mydf = mydf
        
for i in invariant_names:
    inv = build_inv(i)
    setattr(Example,inv.__name__,inv )

for i in property_names:
    prop = build_prop(i)
    setattr(Example, prop.__name__,prop)

if "TARGET" in categorical_names:
    for i in target_property_names:
        prop = build_prop(i)
        setattr(Example, prop.__name__, prop)
else:
    target_invariant = invariant_names.index("TARGET")
print(property_names)

['Sex_female', 'Pclass_3', 'Pclass_1', 'Pclass_2', 'cabin_letter_nan', 'cabin_letter_C', 'cabin_letter_E', 'cabin_letter_G', 'cabin_letter_D', 'cabin_letter_A', 'cabin_letter_B', 'cabin_letter_F', 'cabin_letter_T', 'Embarked_S', 'Embarked_C', 'Embarked_Q', 'Embarked_nan']


Split into training and testing data.

In [37]:
if "TARGET" in categorical_names:
    X_train, X_test = train_test_split(
        my_df.index,
        stratify=my_data["TARGET"],  # stratify on target levels
        train_size=num_train,
        random_state=12345
    )
else:
    X_train, X_test = train_test_split(
        my_df.index,
        train_size=num_train,
        random_state=12345
    )

Create examples for conjecturing.

In [38]:
train_examples = [Example(i, my_df) for i in X_train]
test_examples = [Example(i, my_df) for i in X_test]

Get lists of invariant and property functions.

In [39]:
invariants =[]
for i in invariant_names:
    invariants.append(Example.__dict__[i])
properties=[]
for i in property_names:
    properties.append(Example.__dict__[i])
target_properties=[]
if "TARGET" in categorical_names:
    for i in target_property_names:
        target_properties.append(Example.__dict__[i])


In [40]:
my_df

,Age,Fare,SibSp,Parch,TARGET_0,TARGET_1,TARGET_nan,Sex_female,Sex_male,Sex_nan,...,cabin_letter_D,cabin_letter_E,cabin_letter_F,cabin_letter_G,cabin_letter_T,cabin_letter_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,22.0,7.2500,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
2,38.0,71.2833,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,26.0,7.9250,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
4,35.0,53.1000,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
5,35.0,8.0500,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,27.0,13.0000,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
888,19.0,30.0000,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
889,NaN,23.4500,1,2,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0


Invariant conjecturing - upper and lower bounds.

In [41]:
#define operators for expression tree to build upper bounsand lower bouns for each class
use_operators =  { '-1', '+1', '*2', '/2', '^2', '-()', '1/', 
                  'sqrt', 'ln', 'log10', 'exp', '10^', 'ceil', 
                  'floor', 'abs', '+', '*', 'max', 'min', '-', '/', '^'}

inv_conjectures = []

if "TARGET" in categorical_names:
    for value in target_property_names:
        print(value)
        target_property = Example.__dict__[value]
        my_examples = [example for example in train_examples if target_property(example) == True]
        for inv in invariants:
            sys.stdout.flush()
            inv_of_interest = invariants.index(inv)
            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=True, 
                               time=Integer(5)
                             # ,debug=True,
                             #  verbose=True,
                               ,skips=my_skips
                              )
            convert_conjecture_names(conjs)
            inv_conjectures += conjs

            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=False, 
                               time=Integer(5)
                              ,skips=my_skips)
            convert_conjecture_names(conjs)
            inv_conjectures += conjs
    print(len(inv_conjectures))
    if len(target_property_names) == 1:
        value = target_property_names[0]
        print(value + " False")
        target_property = Example.__dict__[value]
        my_examples = [example for example in train_examples if target_property(example) == False]
        for inv in invariants:
            sys.stdout.flush()
            inv_of_interest = invariants.index(inv)
            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=True, 
                               time=Integer(5)
                             # ,debug=True,
                             #  verbose=True,
                               ,skips=my_skips
                              )
            convert_conjecture_names(conjs)
            inv_conjectures += conjs

            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=False, 
                               time=Integer(5)
                              ,skips=my_skips)
            convert_conjecture_names(conjs)
            inv_conjectures += conjs
else: # target is an invariant
    my_examples = [example for example in train_examples]
    conjs = conjecture(my_examples, 
                       invariants, 
                       target_invariant, 
                       operators=use_operators, 
                       upperBound=True, 
                       time=Integer(5)
                        # ,debug=True,
                        #  verbose=True,
                        ,skips=my_skips)
    convert_conjecture_names(conjs)
    inv_conjectures += conjs
    conjs = conjecture(my_examples, 
                       invariants, 
                       target_invariant, 
                       operators=use_operators,
                       upperBound=False, 
                       time=Integer(5)
                       ,skips=my_skips)
    convert_conjecture_names(conjs)
    inv_conjectures += conjs     
print(len(inv_conjectures))  

for c in inv_conjectures:
    inv_file.write("%s\n" % c)
    inv_file.flush()
inv_file.close()


TARGET_1
28
TARGET_1 False
41


Property conjecturing - sufficient conditions for a categorical target values.  For a binary target, get sufficient conditions for the positive class and necessary conditions for the negative class.

In [42]:
all_properties = ["TARGET"] + properties + inv_conjectures #"TARGET" is just a placeholder
prop_conjs = []
conditions = {}
if "TARGET" in categorical_names:
    for value in target_property_names:
        print(value)
        all_properties[0] = Example.__dict__[value]
        #print(all_properties)
        these_prop_conjs = propertyBasedConjecture(objects=train_examples, # edit here 6/27/23
                                           properties = all_properties,
                                           mainProperty=0,
                                           #verbose=True,
                                           #debug=True,
                                           skips=my_skips)
        conditions[value] = []
        for c in these_prop_conjs: # edit here 6/27/23 just get premises once
            conditions[value].append(get_premise(c, myprint=False))
        prop_conjs += these_prop_conjs
    if len(target_property_names) == 1:
        print(value + " Necessary")
        all_properties[0] = Example.__dict__[value]
        these_prop_conjs = propertyBasedConjecture(objects=train_examples,  # edit here 6/27/23
                                           properties = all_properties,
                                           mainProperty=0,
                                           sufficient=False,
                                           #verbose=True,
                                            #  debug=True,
                                             skips=my_skips)
        conditions["necessary"] = []
        for c in these_prop_conjs:
            conditions["necessary"].append(get_conclusion(c, myprint=False))
        prop_conjs += these_prop_conjs  # edit here 6/27/23
        
for c in prop_conjs:
    prop_file.write("%s\n" % convert_name_back(c.__name__))
    prop_file.flush()
    
prop_file.close()

TARGET_1


/lustre/home/clarson/anaconda3/envs/sage/lib/python3.11/site-packages/sage/misc/functional.py:1209: RuntimeWarning: divide by zero encountered in log
  return ln(args[0], **kwds)
/lustre/home/clarson/anaconda3/envs/sage/lib/python3.11/site-packages/sage/misc/functional.py:1209: RuntimeWarning: invalid value encountered in log
  return ln(args[0], **kwds)


(Sex_female)->(TARGET_1)
(Pclass_2)->(TARGET_1)
(cabin_letter_C)->(TARGET_1)
(~(Fare_leq_10_to_the_power_e_to_the_power_open_bracket_e_to_the_power_SibSp_divided_by_10_to_the_power_Parch_close_bracket))->(TARGET_1)
TARGET_1 Necessary
(TARGET_1)->(Age_leq_2_divided_by_SibSp)
(TARGET_1)->((Fare_leq_maximumopen_bracket_logopen_bracket_Age_close_bracket_or_logopen_bracket__minus_SibSp_close_bracket_close_bracket_squared)->(Sex_female))


In [43]:
len(these_prop_conjs)

2

Apply conjectures to train and test data if target is categorical.

In [44]:
X_train_df = my_df.loc[X_train,property_names+invariant_names]  # drop target and one level for each binary variable
X_test_df = my_df.loc[X_test,property_names+invariant_names]
y_train_df = my_data.loc[X_train,"TARGET"] # get original target, even if it is multiple levels
y_test_df = my_data.loc[X_test, "TARGET"]
if "TARGET" in categorical_names:
    for value in target_property_names:
        for i, condition in enumerate(conditions[value]):
            X_train_df['conj_' + str(i)] = [condition(example) for example in train_examples]
            X_test_df['conj_' + str(i)] = [condition(example) for example in test_examples]
    if len(target_property_names) == 1:
        for i, condition in enumerate(conditions["necessary"]):
            X_train_df['conj_' + str(i)] = [condition(example) for example in train_examples]
            X_test_df['conj_' + str(i)] = [condition(example) for example in test_examples]
        
    X_train_df.head()
    #y_train_df.head()    
    print(conditions)

/lustre/home/clarson/conjecturing/sage/conjecturing.py:279: RuntimeWarning: overflow encountered in scalar power
  return (lambda x: 10**x), 1
/lustre/home/clarson/anaconda3/envs/sage/lib/python3.11/site-packages/sage/misc/functional.py:1209: RuntimeWarning: divide by zero encountered in log
  return ln(args[0], **kwds)
/lustre/home/clarson/anaconda3/envs/sage/lib/python3.11/site-packages/sage/misc/functional.py:1209: RuntimeWarning: invalid value encountered in log
  return ln(args[0], **kwds)


{'TARGET_1': [Sex_female, Pclass_2, cabin_letter_C, ~Fare_leq_10_to_the_power_e_to_the_power_open_bracket_e_to_the_power_SibSp_divided_by_10_to_the_power_Parch_close_bracket], 'necessary': [Age_leq_2_divided_by_SibSp, Fare_leq_maximumopen_bracket_logopen_bracket_Age_close_bracket_or_logopen_bracket__minus_SibSp_close_bracket_close_bracket_squared->Sex_female]}


Calculate support, precision, and lift.

In [45]:
target_property_names

['TARGET_1']

In [46]:
support = []
lift = []
precision = []
if "TARGET" in categorical_names:
    for value in target_property_names:
        print("value: {}".format(value))
        my_function = getattr(Example, value)
        for i, condition in enumerate(conditions[value]):
            print("condition: {}".format(condition))
            num_true = 0
            num_in_class = 0
            num_hit = 0
            for example in test_examples:
                if condition(example) == True:
                    num_true += 1
                    if my_function(example) == True:
                        num_hit += 1
                if my_function(example) == True:
                    num_in_class += 1
            support.append(num_true)
            if num_hit > 0: 
                precision.append(n(num_hit/num_true))
                lift.append(n(num_hit/num_true)/n(num_in_class/len(test_examples)))
            else:
                precision.append(0.0)
                lift.append(0.0)
    if len(target_property_names) == 1:
        for i, condition in enumerate(conditions["necessary"]):
            print("condition: {}".format(condition))
            num_false = 0
            num_in_class = 0
            num_hit = 0
            for example in test_examples:
                if condition(example) == False:
                    num_false += 1
                    if my_function(example) == False:
                        num_hit += 1
                if my_function(example) == False:
                    num_in_class += 1
            support.append(num_false)
            if num_hit > 0: 
                precision.append(n(num_hit/num_false))
                lift.append(n(num_hit/num_false)/n(num_in_class/len(test_examples)))
            else:
                precision.append(0.0)
                lift.append(0.0)
            
results_df = pd.DataFrame({'support':support, 'precision':precision, 'lift':lift})
        
results_df

value: TARGET_1
condition: Sex_female
condition: Pclass_2
condition: cabin_letter_C
condition: ~Fare_leq_10_to_the_power_e_to_the_power_open_bracket_e_to_the_power_SibSp_divided_by_10_to_the_power_Parch_close_bracket


/lustre/home/clarson/conjecturing/sage/conjecturing.py:279: RuntimeWarning: overflow encountered in scalar power
  return (lambda x: 10**x), 1


condition: Age_leq_2_divided_by_SibSp
condition: Fare_leq_maximumopen_bracket_logopen_bracket_Age_close_bracket_or_logopen_bracket__minus_SibSp_close_bracket_close_bracket_squared->Sex_female


,support,precision,lift
0,313,0.741214057507987,1.93198101971756
1,183,0.469945355191257,1.22491673941863
2,58,0.586206896551724,1.52795347888186
3,135,0.600000000000000,1.56390532544379
4,406,0.613300492610837,0.995060283591432
5,294,0.877551020408163,1.42379824858120


In [47]:
support

[313, 183, 58, 135, 406, 294]

In [48]:
precision

[0.741214057507987,
 0.469945355191257,
 0.586206896551724,
 0.600000000000000,
 0.613300492610837,
 0.877551020408163]

In [49]:
lift

[1.93198101971756,
 1.22491673941863,
 1.52795347888186,
 1.56390532544379,
 0.995060283591432,
 1.42379824858120]

In [50]:
conditions

{'TARGET_1': [Sex_female,
  Pclass_2,
  cabin_letter_C,
  ~Fare_leq_10_to_the_power_e_to_the_power_open_bracket_e_to_the_power_SibSp_divided_by_10_to_the_power_Parch_close_bracket],
 'necessary': [Age_leq_2_divided_by_SibSp,
  Fare_leq_maximumopen_bracket_logopen_bracket_Age_close_bracket_or_logopen_bracket__minus_SibSp_close_bracket_close_bracket_squared->Sex_female]}